In [ ]:
! uv pip install agentics-py

import os
import sys
from getpass import getpass

from dotenv import find_dotenv, load_dotenv
import time

CURRENT_PATH = ""

IN_COLAB = "google.colab" in sys.modules
print("In Colab:", IN_COLAB)


if IN_COLAB:
    CURRENT_PATH = "/content/drive/MyDrive/"
    # Mount your google drive
    from google.colab import drive
    drive.mount("/content/drive")
    load_dotenv("/content/drive/MyDrive/.env")
else:
    load_dotenv(find_dotenv())

if not os.getenv("GEMINI_API_KEY"):
    os.environ["GEMINI_API_KEY"] = getpass("Enter your GEMINI_API_KEY:")

begin_time=time.time()

In [ ]:
from typing import Optional, Union, List, Dict, Any
from pydantic import BaseModel, ConfigDict, Field
from pandas import DataFrame

class Text2sqlQuestion(BaseModel): 
    model_config = ConfigDict(arbitrary_types_allowed=True)
    question: Optional[str] =None
    db_id: Optional[str] =None
    query: Optional[str] =None
    reasoning_type: Optional[str] =None
    commonsense_knowledge: Optional[str] =None
    schema: Optional[str] = None
    generated_query: Optional[str] =Field(None, description="The query generated by AI")
    system_output_df: Optional[str] = None
    gt_output_df: Optional[str] = None
    

In [ ]:
import re

def fix_double_quoted_literals(sql: str) -> str:
    """
    Convert double-quoted *literals* to single-quoted strings.
    Keep double-quoted *identifiers* like "MyTable" as-is.

    Heuristic: if the content is a simple identifier ([A-Za-z_][A-Za-z0-9_]*),
    we keep the double quotes; otherwise we treat it as a literal and convert.
    """
    ident_re = re.compile(r'^[A-Za-z_][A-Za-z0-9_]*$')

    def repl(m):
        body = m.group(1).replace('""', '"')  # unescape doubled quotes inside ""
        if ident_re.fullmatch(body):
            # looks like an identifier → leave as "Identifier"
            return f'"{m.group(1)}"'
        # looks like a literal → convert to '...'
        return "'" + body.replace("'", "''") + "'"

    # Match " ... " allowing doubled "" inside
    return re.sub(r'"((?:[^"]|"")*)"', repl, sql)

In [ ]:
from pandas import DataFrame
import aiosqlite
import asyncio

async def async_execute_sql(sql_query: str, db_path:str) -> str:
        try:
            async with aiosqlite.connect(db_path) as db:
                async with db.execute(sql_query.replace("\"","'")) as cursor:
                    columns = [description[0] for description in cursor.description]
                    rows = await asyncio.wait_for(cursor.fetchall(), timeout=10)
                    df = DataFrame(rows, columns=columns)
                    return df.to_json()
        except Exception as e:
            return f"Error: {str(e)}"

In [ ]:
import sqlite3
def get_schema(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    schema_json={}
    for table in tables:

        cursor.execute(f"PRAGMA table_info({table[0]});")
        schema = cursor.fetchall()
        
        schema_json[table[0]] = { col[1] : {"type" : col[2], "notnull" : col[3], "dflt_value": col[4], } for col in schema} 
    return schema_json

res = get_schema("/Users/gliozzo/Data/Text2SQL/Archer/database/bike_1/bike_1.sqlite")
print(res)

In [ ]:

from typing import Set
import pandas as pd


def remove_duplicate_col_df(df):
    return df.loc[:, ~df.columns.duplicated()]


def convert_df_to_set(df, row_invariant=True) -> Set:
    # remove duplicate columns
    df = remove_duplicate_col_df(df)

    if row_invariant:
        return set(
            [
                tuple(sorted(df[c].to_list(), key=lambda x: (x is None, str(x))))
                for c in df.columns.values
            ]
        )
    else:
        return set([tuple(df[c].to_list()) for c in df.columns.values])



def compare_df(gt, predicted, row_invariant=False) -> bool:
    # 1: gt_df is subset of predicted_df
    # 2: df1 == df2
    # 0: otherwise
    if predicted.startswith("Error:") or gt.startswith("Error:"): return 0
    gt_df = pd.read_json(gt)
    predicted_df = pd.read_json(predicted)
    gt_df = gt_df.map(lambda x: float(f"{x:.5f}") if isinstance(x, float) else x)
    predicted_df = predicted_df.map(
        lambda x: float(f"{x:.5f}") if isinstance(x, float) else x
    )

    gt_set = convert_df_to_set(gt_df, row_invariant=row_invariant)
    predicted_set = convert_df_to_set(predicted_df, row_invariant=row_invariant)

    intersec = gt_set & predicted_set
    return (
        1
        if (intersec == gt_set)
        else 1
        if (predicted_set == gt_set)
        else 1
        if (intersec == predicted_set)
        else 0
    )


In [ ]:
from crewai.tools import tool
## Define a Crew AI tool to get news for a given date using the DDGS search engine
@tool("execute_sql_query")
async def execute_sql_query(sql_query:str, db_id:str)-> str:
    """Execute a SQL query against the target db and return the execution results (error or json dataframe)"""
    schema_path = os.path.join("/Users/gliozzo/Data/Text2SQL/Archer/database/", 
                            db_id,db_id+".sqlite" )
    system_output_df= await async_execute_sql(sql_query, schema_path)
    return system_output_df 


In [ ]:
from agentics import Agentics as AG
from agentics.core.llm_connections import get_llm_provider

training = AG.from_jsonl("/Users/gliozzo/Data/Text2SQL/Archer/en_data/dev.json", 
                        jsonl=False, 
                        atype=Text2sqlQuestion,
                        #max_rows=1,
                        )
training.llm=get_llm_provider("watsonx")
training.reasoning=False
training.tools=[execute_sql_query]
training.max_iter=10
training[0].model_dump_json()

In [ ]:
async def get_schema_map(state:Text2sqlQuestion)-> Text2sqlQuestion:
    schema_path = os.path.join("/Users/gliozzo/Data/Text2SQL/Archer/database/", 
                            state.db_id,state.db_id+".sqlite" )
    state.schema=str(get_schema(schema_path))
    print(state.db_id)
    return state

training= await training.amap(get_schema_map)
training.verbose_agent=False
print(training[0].schema)

In [ ]:
training = await training.self_transduction(["question","schema","commonsense_knowledge", "db_id"], 
                            ["generated_query"], 
                            instructions="Generate a SQL query from the input question and target db schema")
print(training[0].generated_query)

In [ ]:
async def execute_query_map(state:Text2sqlQuestion)-> Text2sqlQuestion:
    schema_path = os.path.join("/Users/gliozzo/Data/Text2SQL/Archer/database/", 
                            state.db_id,state.db_id+".sqlite" )
    state.system_output_df= await async_execute_sql(state.generated_query, schema_path)
    state.gt_output_df= await async_execute_sql(state.query, schema_path)
    return state

training= await training.amap(execute_query_map)
print(training[0].gt_output_df)


In [ ]:
training.to_jsonl("/tmp/anker_task.jsonl")

In [ ]:
print(f"task executed in {time.time() - begin_time} seconds")

In [ ]:
training = AG.from_jsonl("/tmp/anker_task.jsonl")
total = 0
for question in training:
    total+= compare_df(question.system_output_df, question.gt_output_df)
    print(question.gt_output_df, question.system_output_df, question.generated_query,compare_df(question.system_output_df, question.gt_output_df) )
print(f"Test size: {len(training.states)}\nExecution Accuracy: {total/len(training.states)}")

In [ ]:
training = AG.from_jsonl("/Users/gliozzo/Data/Text2SQL/Archer/en_data/train.json",jsonl=False)
reasoning_types={}
import random
for question in training:
    for reasoning_type in question.reasoning_type.split(" "):
        if not reasoning_type in reasoning_types: 
            reasoning_types[reasoning_type] = []
        reasoning_types[reasoning_type].append(question)
for reasoning_type in reasoning_types:
    print(random.choice(reasoning_types[reasoning_type]))